[View in Colaboratory](https://colab.research.google.com/github/pmanlukas/colab/blob/master/openapi_classifier.ipynb)

# openAPI spec classifier
This notebook is used to create a first proof of concept on a classifier for openAPI files. The classifier will be based on a neural network based architecture and will be implemented in Tensorflow, SciPy and Keras.

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1buil

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# Create a folder to use for the project data
!mkdir -p /content/drive/openapi-data/

In [0]:
!ls /content/drive/openapi-data/

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [0]:
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive

Selecting previously unselected package liblzo2-2:amd64.
(Reading database ... 19735 files and directories currently installed.)
Preparing to unpack .../liblzo2-2_2.08-1.2_amd64.deb ...
Unpacking liblzo2-2:amd64 (2.08-1.2) ...
Selecting previously unselected package libarchive13:amd64.
Preparing to unpack .../libarchive13_3.2.2-3.1_amd64.deb ...
Unpacking libarchive13:amd64 (3.2.2-3.1) ...
Selecting previously unselected package libarchive-dev:amd64.
Preparing to unpack .../libarchive-dev_3.2.2-3.1_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up liblzo2-2:amd64 (2.08-1.2) ...
Setting up libarchive13:amd64 (3.2.2-3.1) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...


In [0]:
import os
os.chdir("drive/openapi-data")

In [0]:
!ls

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [0]:
import pickle

In [0]:
training_data = dict()

with open('pickle/specs.pkl', 'rb') as handle:
    training_data = pickle.load(handle)

In [0]:
keys = list(training_data.keys())

In [0]:
training_data[keys[0]]

{'basePath': '/forex-quotes',
 'host': '1forge.com',
 'info': {'contact': {'email': 'contact@1forge.com',
   'name': '1Forge',
   'url': 'http://1forge.com'},
  'description': 'Stock and Forex Data and Realtime Quotes',
  'title': '1Forge Finance APIs',
  'version': '0.0.1',
  'x-apisguru-categories': ['financial'],
  'x-logo': {'backgroundColor': '#24292e',
   'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
  'x-origin': [{'format': 'swagger',
    'url': 'http://1forge.com/openapi.json',
    'version': '2.0'}],
  'x-preferred': True,
  'x-providerName': '1forge.com'},
 'paths': {'/quotes': {'get': {'description': 'Get quotes',
    'externalDocs': {'description': 'Find out more',
     'url': 'http://1forge.com/forex-data-api'},
    'responses': {'200': {'description': 'A list of quotes'}},
    'summary': 'Get quotes for all symbols',
    'tags': ['forex', 'finance', 'quotes']}},
  '/symbols': {'get': {'description': 'Symbol List',
    'externalDocs': {'descripti

In [0]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

In [0]:
!ls

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [0]:
labelsP = pd.read_csv("path_labels.csv").values
labelsS  = pd.read_csv("structure_labels.csv").values

In [0]:
labelsS.shape



(1016, 2)

In [0]:
labelsS

array([['1forge.com', 1],
       ['6-dot-authentiqio.appspot.com', 1],
       ['adafruit.com', 2],
       ...,
       ['zoom.us', 4],
       ['zoomconnect.com', 3],
       ['zuora.com', 3]], dtype=object)

## Prepare Labels and Specs for the model
In this part of the notebook, we will create three list. One contains the specs and the other two the labels for paths and for specs. The lists are later used as input for the actual preprocessing and then later the model.

In [0]:
labelsPath = list()
labelsStruct = list()


for labs in labelsS:
  labelsStruct.append(labs[1])

for labp in labelsP:
  labelsPath.append(labp[1])
  
print(str(len(labelsPath)))
print(str(len(labelsStruct)))

#labelsStruct = labelsS["cluster"].tolist()

1016
1016


In [0]:
print(labelsPath)
print(labelsStruct)

[1, 2, 3, 4, 5, 6, 7, 4, 4, 8, 3, 6, 8, 3, 4, 8, 6, 2, 4, 4, 2, 8, 6, 3, 4, 4, 8, 4, 9, 4, 4, 3, 3, 3, 4, 4, 6, 4, 8, 3, 9, 4, 4, 3, 3, 4, 3, 3, 3, 10, 4, 3, 3, 3, 8, 3, 3, 3, 4, 6, 1, 8, 4, 8, 6, 4, 6, 3, 3, 2, 6, 2, 3, 6, 9, 9, 4, 5, 1, 4, 8, 3, 4, 3, 6, 3, 3, 5, 4, 8, 8, 8, 9, 9, 8, 1, 4, 8, 3, 6, 4, 3, 3, 2, 9, 6, 6, 3, 8, 2, 3, 4, 5, 1, 6, 3, 8, 8, 6, 4, 8, 8, 4, 3, 4, 6, 4, 6, 2, 8, 4, 3, 9, 1, 6, 6, 3, 3, 4, 2, 3, 2, 6, 6, 4, 7, 1, 7, 7, 1, 7, 7, 7, 7, 7, 5, 1, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 2, 7, 7, 2, 7, 7, 7, 2, 3, 5, 1, 8, 3, 9, 2, 4, 9, 2, 8, 9, 4, 9, 9, 9, 8, 2, 9, 2, 9, 9, 1, 2, 9, 8, 9, 5, 2, 2, 7, 4, 2, 2, 9, 7, 5, 5, 9, 9, 5, 5, 9, 9, 7, 9, 8, 7, 2, 8, 2, 9, 9, 5, 9, 9, 9, 2, 5, 5, 2, 5, 7, 2, 8, 9, 5, 1, 1, 9, 5, 9, 9, 5, 7, 1, 1, 1, 1, 1, 9, 5, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 7, 5, 5, 9, 4, 8, 9, 4, 2, 2, 4, 4, 6, 1, 5, 1, 3, 9, 8, 5, 7, 7, 9, 9, 8, 9, 7, 9, 4, 3, 9, 3, 4, 3, 2, 8, 5, 4, 8, 6, 2, 9, 4, 4, 5, 5, 7, 9,

In [0]:
specs_json = list()

In [0]:
import json

In [0]:
for spec in training_data:
  strJson = json.dumps(training_data[spec])
  specs_json.append(strJson)

print(str(len(specs_json)))
  
  

1016


In [0]:
specs_json[1]

'{"swagger": "2.0", "schemes": ["https"], "host": "6-dot-authentiqio.appspot.com", "basePath": "/", "info": {"contact": {"email": "hello@authentiq.com", "name": "Authentiq team", "url": "http://authentiq.io/support"}, "description": "Strong authentication, without the passwords.", "license": {"name": "Apache 2.0", "url": "http://www.apache.org/licenses/LICENSE-2.0.html"}, "termsOfService": "http://authentiq.com/terms/", "title": "Authentiq", "version": "6", "x-apisguru-categories": ["security"], "x-logo": {"backgroundColor": "#F26641", "url": "https://api.apis.guru/v2/cache/logo/https_www.authentiq.com_theme_images_authentiq-logo-a-inverse.svg"}, "x-origin": [{"format": "swagger", "url": "https://raw.githubusercontent.com/AuthentiqID/authentiq-docs/master/docs/swagger/issuer.yaml", "version": "2.0"}], "x-preferred": true, "x-providerName": "6-dot-authentiqio.appspot.com"}, "parameters": {"AuthentiqID": {"description": "Authentiq ID to register", "in": "body", "name": "body", "required"

## Preprocessing
In this part of the notebook we are going to preprocess the data itself. The data will be tokenized, preprocessed and stored as Training and Test sets

In [0]:
from keras.preprocessing.text import Tokenizer

Once fit, the Tokenizer provides 4 attributes that you can use to query what has been learned about your documents:

*   word_counts: A dictionary of words and their counts.
*   word_docs: A dictionary of words and how many documents each appeared in.
*   word_index: A dictionary of words and their uniquely assigned integers.
*   document_count:An integer count of the total number of documents that were used to fit the Tokenizer.


[source](https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/)


In [0]:
tokenizer = Tokenizer(num_words=20000,filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')

tokenizer

tokenizer.fit_on_texts(specs_json)

tokenizer.word_index

sequences = tokenizer.texts_to_sequences(specs_json)

In [0]:
print(sequences[0])

[420, 370, 488, 42, 112, 501, 1248, 4020, 340, 476, 193, 476, 7, 86, 112, 2, 3313, 26, 75, 26, 8370, 4020, 92, 3912, 635, 63, 8887, 19, 889, 504, 5663, 19, 382, 2623, 86, 42, 932, 282, 329, 382, 112, 7215, 19, 407, 32, 420, 86, 112, 63, 370, 19, 783, 16, 19, 862, 200, 60, 73, 800, 4020, 46, 2, 46, 4020, 755, 2, 1047, 705, 103, 86, 112, 75, 47, 34, 39, 2, 8, 49, 6, 4020, 94, 46, 4020, 15, 80, 14868, 51, 3912, 4020, 14868, 46, 2, 4369, 49, 755, 2, 1047, 705, 103, 86, 112, 75, 47, 34, 39, 2, 8, 49, 6, 14868, 20, 71, 44, 3, 5, 3, 53, 94, 46, 8, 49, 6, 14868, 15, 95, 441, 550, 2527, 67, 4020, 51, 3912, 4020]


In [0]:
with open('pickle/processed_list.pickle', 'wb') as handle:
    pickle.dump(sequences, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
wordindex = tokenizer.word_index

In [0]:
with open('pickle/word_index.pickle', 'wb') as handle:
    pickle.dump(wordindex, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
indexkeys = list(tokenizer.word_index.keys())
print(indexkeys[0:100])

['the', 'description', 'type', 'ref', 'string', 'of', 'name', 'a', 'in', 'n', 'definitions', 'to', 'parameters', 'required', 'for', 'true', 'code', 'is', 'x', 'schema', 'properties', 'p', 'id', 'this', 'object', 'and', 'that', 'query', 'be', 'or', 'false', 'format', 'if', 'responses', 'integer', 'an', 'value', '.', '200', 'amz', 'you', 'https', 'not', 'items', 'path', 'get', 'api', 'with', 'list', 'number', 'tags', 'default', 'array', 'body', 'date', 'are', 'operationid', 'request', 'can', 'application', 'by', 'resource', 'version', 'user', 'li', 'on', 'time', 'error', 'response', 'token', 'example', 'specified', 'json', 'from', 'data', 'will', 'key', 'as', '1', 'all', 'only', 'status', 'set', 'boolean', 'enum', 'url', 'when', 'ms', 'use', 'information', 'subscriptions', 'title', 'security', 'summary', 'which', '0', 'used', 'limit', 'was', 'readonly']


In [0]:
sequences_padded = pad_sequences(sequences)

In [0]:
len(sequences_padded[3])

378739

In [0]:
with open('pickle/sequence_padded.pickle', 'wb') as handle:
    pickle.dump(sequences_padded, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Model Architecture
In this section of the notebook the actual deep learning model is designed and the architecture presented before the actual training of it will take place.